In [38]:
from langchain.schema import HumanMessage, SystemMessage
import textwrap
import os

from langchain_core.messages.utils import count_tokens_approximately
from langchain_openai import OpenAIEmbeddings

from openai import OpenAI
from langchain_chroma import Chroma

from backend import ROOT_DIR


CHROMA_PATH = f"{ROOT_DIR}/chroma_langchain_db/text-embedding-3-small"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
question = "How many days in advance to ask for adoption leave?"

question = "How much do I still get paid if I go on maternity leave?"

docs = db.similarity_search(question, k=3)

In [39]:
for doc in docs:
    print(doc.page_content)
    print('\n')
    print(doc.metadata)
    print("-" * 100)

iii.for the next 13 weeks, the employee will receive any Statutory Maternity Pay or maternity allowance that they are entitled to under the statutory scheme. iv.For the final 13 weeks, the employee will receive no pay

19. Can I change how this is paid?

Yes, section 15.21 of the handbook states: by prior agreement with the employer, occupational maternity pay may be paid in a different way, for example a combination of full pay and half pay, or a fixed amount spread equally over the maternity leave period. Where occupational maternity pay has been paid in a different way and the employee subsequently choses to access shared parental leave and pay, the employer may need to recalculate payments to ensure that there has not been any over or underpayment of entitlements.

Calculation of maternity/adoption/shared parental pay

20. How is my maternity pay calculated?


{'filename': 'W19-Leave-Policy-Chapter-4-Fertility-Pregnancy-and-Maternity-Additions-to-App-C-Dec-2024-V2.1.pdf', 'pdf_hash

In [40]:
context_texts = [
    [
        f"Document: {doc.metadata['filename']}, Page: {doc.metadata['page_number']}",
        f"Content: {doc.page_content}",
    ]
    for doc in docs
]

context = '\n\n'.join(f"{doc[0]}\n{doc[1]}" for doc in context_texts)

system_prompt = (
    "You are an HR assistant for NHS staffs."
    "Answer the question based solely on the provided policy context, if multiple documents are relevant, provide a summary of relevant information."
    "If the context does not contain the answer, reply 'I couldn't find the relevant information.' instead of inventing one."
    "Please state the answer by adhering closely to the original text, and do not generate any additional information."
    "If sensible, provide users with some extra information that is useful, note that this also has to be based on the context ONLY."
    "At the end of your answer, cite the page number and document name of the text that you used to answer the question."
)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=textwrap.dedent(f"""
        Context:
        {context}

        Question: {question}
    """)),
]

print(f"Tokens in context: {count_tokens_approximately(messages)}")
# calculate how many tokens are in the query

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo")

from langchain.chat_models import init_chat_model

model = init_chat_model(
    OPENAI_MODEL,
    model_provider="openai",
    api_key=OPENAI_API_KEY,
)

Tokens in context: 995


In [41]:
response = model.invoke(messages)

In [42]:
response.content

'If you go on maternity leave, your pay will be calculated as follows:  \n\n- For the first eight weeks of absence, you will receive full pay, less any Statutory Maternity Pay or maternity allowance.\n- For the next 18 weeks, you will receive half of full pay, plus any Statutory Maternity Pay or maternity allowance, as long as the total does not exceed full pay.\n- For the following 13 weeks after that, you will receive any Statutory Maternity Pay or maternity allowance that you are entitled to under the statutory scheme.\n- For the final 13 weeks, you will receive no pay (W19-Leave-Policy-Chapter-4-Fertility-Pregnancy-and-Maternity-Additions-to-App-C-Dec-2024-V2.1.pdf, Page: 8; 7).'

In [1]:
from backend.retrieval.retrieval import get_answer

answer, trace_dict = get_answer("How many days in advance to ask for adoption leave?", trace=True)


[Document(metadata={'filename': 'Managers-and-Staff-Policy-Handbook-2024-W100.pdf', 'page_number': 37, 'pdf_hash': 'c7ed49dd143740536120374bc7ba05b76e00ab485358346c33195b1032fe79e1'}, page_content='Policy implementation:\n\nAdoption leave is the period of absence from work before and after the adoption of a child. A ‘matching certificate’ from the adoption agency is evidence of the entitlement to take Adoption Leave. Employees must notify the manager in writing using the Adoption Leave Form within 7 days of being notified by the adoption agency having been matched with a child for adoption or by the 15th week before the baby’s due date if applying via a surrogacy arrangement. Employees are entitled to take 52 weeks’ adoption leave. Employees are required to give at least 8 weeks’ notice if they wish to return to work before the agreed return date.'), Document(metadata={'page_number': 5, 'pdf_hash': '45ca58270db5708d0be625c6f28f24bf5c1541e94c3ebb741a03978314f8e419', 'filename': 'W19-Lea

In [2]:
answer

'You must notify your Manager in writing using the Adoption Leave Form within 7 days of being notified by your adoption agency that you have been matched with a child for adoption, unless this is not reasonably practicable (W19-Leave-Policy-Chapter-1-Adoption-Leave-Procedure-Amends-April-2024-v2.pdf, Page: 3).'

In [3]:
trace_dict

{'question': 'How many days in advance to ask for adoption leave?',
 'retrieved_docs': [{'page_content': 'Policy implementation:\n\nAdoption leave is the period of absence from work before and after the adoption of a child. A ‘matching certificate’ from the adoption agency is evidence of the entitlement to take Adoption Leave. Employees must notify the manager in writing using the Adoption Leave Form within 7 days of being notified by the adoption agency having been matched with a child for adoption or by the 15th week before the baby’s due date if applying via a surrogacy arrangement. Employees are entitled to take 52 weeks’ adoption leave. Employees are required to give at least 8 weeks’ notice if they wish to return to work before the agreed return date.',
   'metadata': {'filename': 'Managers-and-Staff-Policy-Handbook-2024-W100.pdf',
    'page_number': 37,
    'pdf_hash': 'c7ed49dd143740536120374bc7ba05b76e00ab485358346c33195b1032fe79e1'}},
  {'page_content': 'You can choose to sta